In [57]:
import numpy as np
import sklearn

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import ShuffleSplit, GridSearchCV, train_test_split
from sklearn import svm, metrics, preprocessing
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [58]:
# Importamos los datos solo con las características más importantes

X_reduc2=np.load("./datos_mfcc_pca99_X_reducido2.npy")
y_reduc=np.load("./datos_mfcc_pca99_y_reducido.npy")

In [59]:
print(X_reduc2.shape)

(360, 44)


In [60]:
print(y_reduc.shape)

(360,)


In [61]:
# Ahora ya podemos dividir nuestro conjunto reducido en entrenamiento y test, y probar algunos métodos Kernel.
X_reduc2= preprocessing.StandardScaler().fit_transform(X_reduc2) 
Xtr, Xts, ytr, yts = train_test_split(X_reduc2, y_reduc, test_size=0.3, shuffle=True, random_state=0)


In [62]:
# Aplicamos un Random Forest con todos los parámetros por defecto.

rf = RandomForestClassifier(max_depth=8)   # Lo dejamos todo por defecto a ver qué tal funciona.
rf.fit(Xtr,ytr)

preds_test = rf.predict(Xts)
print('Res Random Forest:', metrics.classification_report(yts, preds_test))

Res Random Forest:               precision    recall  f1-score   support

           0       0.53      0.53      0.53        17
           1       0.57      0.60      0.59        20
           2       0.54      0.35      0.42        20
           3       0.70      0.64      0.67        22
           4       0.58      0.61      0.59        18
           5       0.39      0.64      0.48        11

    accuracy                           0.56       108
   macro avg       0.55      0.56      0.55       108
weighted avg       0.57      0.56      0.55       108



In [63]:
# Mostramos el acierto en entrenamiento y en test del árbol final con el que nos hemos quedado.

print("OA train %0.2f" %rf.score(Xtr, ytr))
print("OA test %0.2f" %rf.score(Xts, yts))

OA train 0.98
OA test 0.56


In [64]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

Vemos que mejoramos los resultados, pero solamente un poco. Probamos a cambiar los parámetros y añadir más árboles.

In [76]:
# Aplicamos un Random Forest cambiando los parámetros por defecto.

T=10000  # Número de árboles

rf2 = RandomForestClassifier(n_estimators=T, max_depth=8)   
rf2.fit(Xtr,ytr)

preds_test2 = rf2.predict(Xts)
print('Res Random Forest:', metrics.classification_report(yts, preds_test2))

Res Random Forest:               precision    recall  f1-score   support

           0       0.54      0.41      0.47        17
           1       0.70      0.70      0.70        20
           2       0.67      0.60      0.63        20
           3       0.80      0.73      0.76        22
           4       0.65      0.61      0.63        18
           5       0.35      0.64      0.45        11

    accuracy                           0.62       108
   macro avg       0.62      0.61      0.61       108
weighted avg       0.64      0.62      0.63       108



In [77]:
# Mostramos el acierto en entrenamiento y en test del árbol final con el que nos hemos quedado.

print("OA train %0.2f" %rf2.score(Xtr, ytr))
print("OA test %0.2f" %rf2.score(Xts, yts))

OA train 0.99
OA test 0.62


Parece que mejoramos algo más el rendimiento con el Random Forest que con el bagging simplemente. En entrenamiento tenemos muy buenos resultados y en test hemos avanzado un poco más.

Podemos probar cambiando el criterio de decisión. El que viene por defecto es el de la impureza de Gini. Probamos con el de la entropía para ver si mejoran los resultados.

In [70]:
# Aplicamos un Random Forest cambiando los parámetros por defecto.

T=10000  # Número de árboles

rf3 = RandomForestClassifier(n_estimators=T, max_depth=6, criterion='entropy')   
rf3.fit(Xtr,ytr)

preds_test3 = rf3.predict(Xts)
print('Res Random Forest:', metrics.classification_report(yts, preds_test3))

Res Random Forest:               precision    recall  f1-score   support

           0       0.58      0.41      0.48        17
           1       0.69      0.55      0.61        20
           2       0.57      0.40      0.47        20
           3       0.73      0.73      0.73        22
           4       0.53      0.56      0.54        18
           5       0.32      0.73      0.44        11

    accuracy                           0.56       108
   macro avg       0.57      0.56      0.55       108
weighted avg       0.59      0.56      0.56       108



In [71]:
# Mostramos el acierto en entrenamiento y en test del árbol final con el que nos hemos quedado.

print("OA train %0.2f" %rf3.score(Xtr, ytr))
print("OA test %0.2f" %rf3.score(Xts, yts))

OA train 0.92
OA test 0.56


En este caso parece que el mejor criterio a la hora de decidir por qué característica hacer la división, es el de la impureza de Gini ya que nos da mejores resultados.